# 高度なファインチューニング実習

このノートブックでは、LLMのファインチューニングの高度なテクニックを学習します。

## 学習目標
- カスタムデータセットでのファインチューニング
- 異なるファインチューニング手法の比較
- ハイパーパラメータの最適化
- モデルの評価と分析
- 実用的なアプリケーションへの応用

## 前提条件
- LLM基礎実習の完了
- PyTorchとTransformersの基本理解
- 機械学習の基礎知識

## 注意事項
- このノートブックは計算集約的です
- GPUの使用を強く推奨します
- 実行時間は数時間に及ぶ場合があります


## 1. 環境セットアップとライブラリのインポート


## 2. カスタムデータセットの作成


In [ ]:
class CustomDataset(Dataset):
    """カスタムデータセットクラス"""
    
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # テキストの前処理
        if isinstance(text, str):
            # 単純なテキストの場合
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )
        else:
            # 構造化されたテキストの場合
            formatted_text = self.format_text(text)
            encoding = self.tokenizer(
                formatted_text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
    
    def format_text(self, text_dict):
        """構造化されたテキストをフォーマット"""
        if 'instruction' in text_dict and 'response' in text_dict:
            return f"Instruction: {text_dict['instruction']}\\nResponse: {text_dict['response']}"
        elif 'context' in text_dict and 'question' in text_dict:
            return f"Context: {text_dict['context']}\\nQuestion: {text_dict['question']}"
        else:
            return str(text_dict)

def create_sample_datasets():
    """サンプルデータセットの作成"""
    
    # 1. 感情分析データセット
    sentiment_data = {
        'texts': [
            "I love this product! It's amazing!",
            "This is terrible. I hate it.",
            "The weather is okay today.",
            "Fantastic! Best experience ever!",
            "I'm so disappointed with this service.",
            "It's fine, I guess.",
            "Outstanding quality and great value!",
            "Worst purchase I've ever made.",
            "Good product, would recommend.",
            "Excellent service and fast delivery!"
        ],
        'labels': [1, 0, 1, 1, 0, 1, 1, 0, 1, 1]  # 0: negative, 1: positive
    }
    
    # 2. 質問応答データセット
    qa_data = {
        'texts': [
            {
                'context': 'Machine learning is a subset of artificial intelligence.',
                'question': 'What is machine learning?'
            },
            {
                'context': 'Python is a popular programming language for data science.',
                'question': 'Which language is good for data science?'
            },
            {
                'context': 'Deep learning uses neural networks with multiple layers.',
                'question': 'How does deep learning work?'
            }
        ],
        'labels': [0, 1, 2]  # カテゴリ分類
    }
    
    # 3. テキスト生成データセット
    generation_data = {
        'texts': [
            {
                'instruction': 'Write a haiku about artificial intelligence',
                'response': 'Silent circuits hum\\nLearning patterns in the void\\nMind awakens new'
            },
            {
                'instruction': 'Explain quantum computing simply',
                'response': 'Quantum computing uses quantum bits that can exist in multiple states simultaneously, allowing for parallel processing.'
            },
            {
                'instruction': 'Create a recipe for chocolate cake',
                'response': 'Mix flour, sugar, cocoa powder, baking powder, and salt. Add eggs, milk, oil, and vanilla. Bake at 350°F for 30 minutes.'
            }
        ],
        'labels': [0, 1, 2]  # タスクタイプ
    }
    
    return sentiment_data, qa_data, generation_data

# データセットの作成
sentiment_data, qa_data, generation_data = create_sample_datasets()

print("📊 サンプルデータセットが作成されました")
print(f"感情分析データ: {len(sentiment_data['texts'])}件")
print(f"質問応答データ: {len(qa_data['texts'])}件")
print(f"テキスト生成データ: {len(generation_data['texts'])}件")


In [ ]:
# 必要なライブラリのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets accelerate evaluate scikit-learn pandas numpy matplotlib seaborn tqdm wandb
!pip install peft bitsandbytes  # LoRA用のライブラリ

# ライブラリのインポート
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset as HFDataset, load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, classification_report
import json
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 環境設定
plt.style.use('default')
sns.set_palette("husl")

print("✅ ライブラリのインポートが完了しました")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
